In [1]:
import pandas as pd
import numpy as np

In [120]:
# generating pair-wise comparision based on user-preference
user_data = pd.read_csv('weight1.csv')

In [121]:
user_data

,Unnamed: 0,Size,Shape,Texture,Vascular
0,Size,1,6,3,8
1,Shape,1/6,1,1/2,4
2,Texture,1/3,2,1,3
3,Vascular,1/8,1/4,1/3,1


In [122]:
for col in user_data.columns:
    user_data.rename(columns = {col:col.strip()},inplace=True)

In [123]:
user_data.drop(columns=['Unnamed: 0'],inplace=True)

In [124]:
# calculating the float values
def division(x):
    try:
        if '/' in x:
            parts = x.split('/')
            return int(parts[0]) / int(parts[1])
        else:
            return int(x)
    except:
        return x
def float_values(df):
    try:
        df.drop(columns=['Pair-wise comp'],inplace=True)
    except:
        df = df
    for col in df.columns:
        df[col] = df[col].apply(lambda x:round(division(x),2))
    return df

In [125]:
def normalization(df):
    for col in df.columns:
        sum_ = round(df[col].sum(),2)
        df[col] = df[col].apply(lambda x:round(x/sum_,2))
    return df

In [126]:
def sum_of_rows(df):
    l=[]
    for i in range(df.shape[0]):
        l.append(round(sum(df.loc[i,:]),2))
    return l

In [127]:
def criterion_weight(df):
    l=[]
    for i in range(df.shape[0]):
        l.append(round(sum(df.loc[i,:])/df.shape[1],2))
    return l

In [128]:
def consistency_mat(caldf,l):
    for i,col in enumerate(caldf.columns):
        caldf[col] = caldf[col].apply(lambda x :l[i]*x)
    return caldf

In [129]:
def weighted_mat(df,cw):
    l = sum_of_rows(df)
    df['Weighted_sum_value'] = pd.Series(l)
    df['Criteria_weight'] = pd.Series(cw)
    df['result']  = df['Weighted_sum_value']/df['Criteria_weight']
    return df

In [130]:
cal_df = float_values(user_data)
cal_df

,Size,Shape,Texture,Vascular
0,1.00,6.00,3.00,8
1,0.17,1.00,0.50,4
2,0.33,2.00,1.00,3
3,0.12,0.25,0.33,1


In [131]:
cal_df_n = cal_df.copy() 
norm_df = normalization(cal_df)
norm_df

,Size,Shape,Texture,Vascular
0,0.62,0.65,0.62,0.50
1,0.10,0.11,0.10,0.25
2,0.20,0.22,0.21,0.19
3,0.07,0.03,0.07,0.06


In [132]:
cal_df_n

,Size,Shape,Texture,Vascular
0,1.00,6.00,3.00,8
1,0.17,1.00,0.50,4
2,0.33,2.00,1.00,3
3,0.12,0.25,0.33,1


In [133]:
cw = criterion_weight(norm_df)
cw

[0.6, 0.14, 0.21, 0.06]

In [134]:
consistency_df = consistency_mat(cal_df_n,cw)
consistency_df

,Size,Shape,Texture,Vascular
0,0.600,0.840,0.6300,0.48
1,0.102,0.140,0.1050,0.24
2,0.198,0.280,0.2100,0.18
3,0.072,0.035,0.0693,0.06


In [135]:
result_df = weighted_mat(consistency_df,cw)
result_df

,Size,Shape,Texture,Vascular,Weighted_sum_value,Criteria_weight,result
0,0.600,0.840,0.6300,0.48,2.55,0.60,4.250000
1,0.102,0.140,0.1050,0.24,0.59,0.14,4.214286
2,0.198,0.280,0.2100,0.18,0.87,0.21,4.142857
3,0.072,0.035,0.0693,0.06,0.24,0.06,4.000000


In [136]:
cal_df.shape[1]

4

In [137]:
n=cal_df.shape[1]
lam_max = np.array(result_df['result']).mean()
consistency_index = (lam_max - n)/(n-1)

In [138]:
lam_max

4.151785714285714

In [139]:
(lam_max - n)/3

0.05059523809523814

In [140]:
lam_max

4.151785714285714

In [141]:
consistency_index

0.05059523809523814

In [142]:
consistent_Ratio = consistency_index/0.90

In [143]:
round(consistent_Ratio,2)

0.06